In [2]:
import os
import pandas as pd
#!pip install pyarrow
import pyarrow.parquet as pq

In [3]:
os.getcwd()
folder_path = 'taxi'
# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.parquet'):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Read Parquet file into a DataFrame
        df = pq.read_table(file_path).to_pandas()
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
trips = pd.concat(dfs, ignore_index=True)

In [4]:
trips.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
airport_fee                     float64
dtype: object

In [5]:
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1.0,3.40,1.0,N,140,238,1,21.9,3.50,0.5,6.70,0.0,1.0,33.60,2.5,0.00,NaN
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0.0,3.40,1.0,N,50,151,1,15.6,3.50,0.5,3.00,0.0,1.0,23.60,2.5,0.00,NaN
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1.0,10.20,1.0,N,138,97,1,40.8,7.75,0.5,10.00,0.0,1.0,60.05,0.0,1.75,NaN
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3.0,9.83,1.0,N,100,244,1,39.4,1.00,0.5,8.88,0.0,1.0,53.28,2.5,0.00,NaN
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1.0,1.17,1.0,N,137,234,1,9.3,1.00,0.5,0.72,0.0,1.0,15.02,2.5,0.00,NaN


In [6]:
trips.shape

(77966324, 20)

In [7]:
trips[["tpep_pickup_datetime","tpep_dropoff_datetime"]].max()

tpep_pickup_datetime    2024-01-03 19:42:57
tpep_dropoff_datetime   2024-01-03 20:15:55
dtype: datetime64[ns]

In [8]:
trips[["tpep_pickup_datetime","tpep_dropoff_datetime"]].min()

tpep_pickup_datetime    2001-01-01 00:03:14
tpep_dropoff_datetime   1970-01-20 10:16:32
dtype: datetime64[ns]

In [11]:
trips=  trips[(trips['tpep_dropoff_datetime'].dt.year<=2023) & (trips['tpep_pickup_datetime'].dt.year<=2023)&(trips['tpep_dropoff_datetime'].dt.year>=2022)&(trips['tpep_pickup_datetime'].dt.year>=2022)]

In [10]:
#data cleaning
#1. passanger count
#trips.passenger_count.describe()
trips=trips[(trips['passenger_count']>0)&(trips['passenger_count']<=6)]
#2. trip distance 
trips=trips[(trips['trip_distance']>=0)]
#3. trip fare
#we only focus on RatecodeID=1 which is the Standard Rate, no negotiate prices and other influencer
trips=trips[trips['RatecodeID']==1]
#We only keep orders paid by cash and credit card
trips=trips[(trips['payment_type']==1)|(trips['payment_type']==2)]

In [12]:
trips.groupby('PULocationID').size().sort_values(ascending=False)

PULocationID
237    3442895
161    3092792
236    3037789
162    2468472
186    2440742
        ...   
27          11
99          10
5            9
84           9
204          8
Length: 262, dtype: int64

In [13]:
#p_zones = [237, 161, 236, 162,138]
p_zones=[237,161,236]
trips.query(trips['PULocationID'] in p_zones)

In [14]:
start_date_2022 = pd.to_datetime('2022-01-01 00:00')
end_date_2023 = pd.to_datetime('2024-01-01 00:00')
timeline_2023 = pd.date_range(start=start_date_2022, end=end_date_2023, freq='30min')


In [15]:
result = pd.DataFrame(columns=['tpep_pickup_datetime', 'order_cnt', 'PULocationID'])
for idx, time in enumerate(timeline_2023[:-1]):
    next_time = timeline_2023[idx + 1]
    orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (trips['tpep_pickup_datetime'] < next_time)]
    order_counts = orders_in_interval.groupby('PULocationID').size().reset_index(name='order_cnt')
    order_counts['tpep_pickup_datetime'] = time
    result = pd.concat([result, order_counts], ignore_index=True)
    if(idx%10==0):
        print(f"{idx}done")

0done
10done
20done
30done
40done
50done
60done
70done
80done
90done
100done
110done
120done
130done
140done
150done
160done
170done
180done
190done
200done
210done
220done
230done
240done
250done
260done
270done
280done
290done
300done
310done
320done
330done
340done
350done
360done
370done
380done
390done
400done
410done
420done
430done
440done
450done
460done
470done
480done
490done
500done
510done
520done
530done
540done
550done
560done
570done
580done
590done
600done
610done
620done
630done
640done
650done
660done
670done
680done
690done
700done
710done
720done
730done
740done
750done
760done
770done
780done
790done
800done
810done
820done
830done
840done
850done
860done
870done
880done
890done
900done
910done
920done
930done
940done
950done
960done
970done
980done
990done
1000done
1010done
1020done
1030done
1040done
1050done
1060done
1070done
1080done
1090done
1100done
1110done
1120done
1130done
1140done
1150done
1160done
1170done
1180done
1190done
1200done
1210done
1220done
1230

9230done
9240done
9250done
9260done
9270done
9280done
9290done
9300done
9310done
9320done
9330done
9340done
9350done
9360done
9370done
9380done
9390done
9400done
9410done
9420done
9430done
9440done
9450done
9460done
9470done
9480done
9490done
9500done
9510done
9520done
9530done
9540done
9550done
9560done
9570done
9580done
9590done
9600done
9610done
9620done
9630done
9640done
9650done
9660done
9670done
9680done
9690done
9700done
9710done
9720done
9730done
9740done
9750done
9760done
9770done
9780done
9790done
9800done
9810done
9820done
9830done
9840done
9850done
9860done
9870done
9880done
9890done
9900done
9910done
9920done
9930done
9940done
9950done
9960done
9970done
9980done
9990done
10000done
10010done
10020done
10030done
10040done
10050done
10060done
10070done
10080done
10090done
10100done
10110done
10120done
10130done
10140done
10150done
10160done
10170done
10180done
10190done
10200done
10210done
10220done
10230done
10240done
10250done
10260done
10270done
10280done
10290done
10300do

17500done
17510done
17520done
17530done
17540done
17550done
17560done
17570done
17580done
17590done
17600done
17610done
17620done
17630done
17640done
17650done
17660done
17670done
17680done
17690done
17700done
17710done
17720done
17730done
17740done
17750done
17760done
17770done
17780done
17790done
17800done
17810done
17820done
17830done
17840done
17850done
17860done
17870done
17880done
17890done
17900done
17910done
17920done
17930done
17940done
17950done
17960done
17970done
17980done
17990done
18000done
18010done
18020done
18030done
18040done
18050done
18060done
18070done
18080done
18090done
18100done
18110done
18120done
18130done
18140done
18150done
18160done
18170done
18180done
18190done
18200done
18210done
18220done
18230done
18240done
18250done
18260done
18270done
18280done
18290done
18300done
18310done
18320done
18330done
18340done
18350done
18360done
18370done
18380done
18390done
18400done
18410done
18420done
18430done
18440done
18450done
18460done
18470done
18480done
18490done


25700done
25710done
25720done
25730done
25740done
25750done
25760done
25770done
25780done
25790done
25800done
25810done
25820done
25830done
25840done
25850done
25860done
25870done
25880done
25890done
25900done
25910done
25920done
25930done
25940done
25950done
25960done
25970done
25980done
25990done
26000done
26010done
26020done
26030done
26040done
26050done
26060done
26070done
26080done
26090done
26100done
26110done
26120done
26130done
26140done
26150done
26160done
26170done
26180done
26190done
26200done
26210done
26220done
26230done
26240done
26250done
26260done
26270done
26280done
26290done
26300done
26310done
26320done
26330done
26340done
26350done
26360done
26370done
26380done
26390done
26400done
26410done
26420done
26430done
26440done
26450done
26460done
26470done
26480done
26490done
26500done
26510done
26520done
26530done
26540done
26550done
26560done
26570done
26580done
26590done
26600done
26610done
26620done
26630done
26640done
26650done
26660done
26670done
26680done
26690done


33900done
33910done
33920done
33930done
33940done
33950done
33960done
33970done
33980done
33990done
34000done
34010done
34020done
34030done
34040done
34050done
34060done
34070done
34080done
34090done
34100done
34110done
34120done
34130done
34140done
34150done
34160done
34170done
34180done
34190done
34200done
34210done
34220done
34230done
34240done
34250done
34260done
34270done
34280done
34290done
34300done
34310done
34320done
34330done
34340done
34350done
34360done
34370done
34380done
34390done
34400done
34410done
34420done
34430done
34440done
34450done
34460done
34470done
34480done
34490done
34500done
34510done
34520done
34530done
34540done
34550done
34560done
34570done
34580done
34590done
34600done
34610done
34620done
34630done
34640done
34650done
34660done
34670done
34680done
34690done
34700done
34710done
34720done
34730done
34740done
34750done
34760done
34770done
34780done
34790done
34800done
34810done
34820done
34830done
34840done
34850done
34860done
34870done
34880done
34890done


In [16]:
result.to_csv('result.csv')

In [17]:
result.head(10)

,tpep_pickup_datetime,order_cnt,PULocationID
0,2022-01-01 00:00:00,16,138
1,2022-01-01 00:00:00,55,161
2,2022-01-01 00:00:00,38,162
3,2022-01-01 00:00:00,40,236
4,2022-01-01 00:00:00,63,237
5,2022-01-01 00:30:00,29,138
6,2022-01-01 00:30:00,74,161
7,2022-01-01 00:30:00,55,162
8,2022-01-01 00:30:00,60,236
9,2022-01-01 00:30:00,109,237


In [19]:
per=0.8
for region in p_zones:
    df=result[result['PULocationID']==region]
    df=df.sort_values(by='tpep_pickup_datetime')
    train_size=int(len(df)*per)
    train_data,test_data = df[:train_size],df[train_size:]
    file_name_train='train_data'+str(region)+'.csv'
    file_name_test='test_data'+str(region)+'.csv'
    train_data.to_csv(file_name_train,index=False)
    test_data.to_csv(file_name_test,index=False)